In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.
The file _gams_py_gdb5.gdx is still active and was not deleted.
The file _gams_py_gdb6.gdx is still active and was not deleted.


*Load specific modules used here:*

In [2]:
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
import mGovernment
from gmsPython import nestingTree
from valueShares import nestedShares_noOutputs

# Government modules

*Load test data:*

In [3]:
t0 = 2019
name = 'TestModelData' #
db = GpyDB(os.path.join(d['data'], name), name = 'test', ws = d['work'])
ws = db.ws

Subset data to only include government sector:

In [4]:
AggDB.subsetDB(db, db('s_G'))

Note: This also removes tax income from other sectors. This does not break with the basic models though, but we lose out on some part of the government revenue here.

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [5]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

## 1. ```GovNCES```

In [6]:
db_NCES = db.copy()
db_i = db_NCES

*Set up nesting structure from nesting tree:*

In [7]:
nest = nestingTree.Tree('G', tree = db_i('nestG').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'G', trees = {t.name: t for t in [nest]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

*Get starting values for $\mu$ using value shares:*

In [8]:
v = nestedShares_noOutputs(nestAgg, ws = ws)
db_vs = v(db) # one-liner that returns solution database

Use share parameters and values to get initial values for intermediates:

In [9]:
db_i.aom(db_vs('mu').xs(t0), name = 'mu')
db_i.aom(adj.rc_pd(db_vs('vD'), nestAgg.get('int').union(nestAgg.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Initialize model instance:
* ```self.taxInstr```: Can be lumpsum tax (```lumpSum```) or input taxes (```tauD```). Specifies how total government transfers are targeted in calibration.
* ```self.incInstr```: Can be $j$-term (```jTerm```) or initial assets (```vA0```).  Specifies how to ensure that law of motion for assets clears in initial period where all income streams are fixed. 

In [10]:
m_NCES = mGovernment.GovNCES(nestAgg, partial = True, properties = {'taxInstr': 'tauLump', 'incInstr': 'jTerm'})
m_i = m_NCES

Add data and initialize other stuff with default options:

In [11]:
m_i.initStuff(db = db_i)

Set value of assets to zero (hand-to-mouth like behavior):

In [12]:
m_i.db['vA'] = m_i.db('vA')*0
m_i.db.mergeInternal()

Solve calibration:

In [14]:
solve_NCES_Calib = m_i.solve(state = 'C')
solve_i = solve_NCES_Calib

Add solution to main database, remove init methods (we want to use this solution as initial value):

In [15]:
[m_i.db.__setitem__(k, solve_i[k]) for k in m_i.db.getTypes(['var']) if k in solve_i.symbols]; # use solution database
m_i.db.mergeInternal()

Test baseline state:

In [16]:
solve_NCES = m_i.solve(state = 'B') # solve baseline state
solve_i = solve_NCES